In [1]:
import match_service_pb2
import match_service_pb2_grpc
import grpc

from google.cloud import aiplatform_v1

In [2]:
REGION = "us-west1"
PEERING_RANGE_NAME = "spotifind-vpc-peering-range2"
location = "projects/841506577075/locations/{}".format(REGION)
aiplatform_endpoint = "{}-aiplatform.googleapis.com".format(REGION)

In [3]:
# (IndexEndpointService) get ANN index endpoint for current region (~us-west1c/us-central1a)
index_endpoint_service_client = aiplatform_v1.IndexEndpointServiceClient(client_options=dict(api_endpoint=aiplatform_endpoint))

list_index_endpoints_request = aiplatform_v1.ListIndexEndpointsRequest(parent=location)

In [4]:
pager_result = index_endpoint_service_client.list_index_endpoints(request=list_index_endpoints_request)

In [5]:
spotifind_index_endpoint = pager_result.index_endpoints[0]
spotifind_index_endpoint

name: "projects/841506577075/locations/us-west1/indexEndpoints/5298379408485122048"
display_name: "index_endpoint_for_spotifind_demo2"
deployed_indexes {
  id: "spotifind_ann_index-deployed"
  index: "projects/841506577075/locations/us-west1/indexes/8335635144681455616"
  display_name: "spotifind_ann_index-deployed"
  create_time {
    seconds: 1655663657
    nanos: 887493000
  }
  private_endpoints {
    match_grpc_address: "172.24.2.5"
  }
  index_sync_time {
    seconds: 1655683243
    nanos: 368194000
  }
  automatic_resources {
    min_replica_count: 2
    max_replica_count: 2
  }
  reserved_ip_ranges: "spotifind-vpc-peering-range2"
  deployment_group: "default"
}
etag: "AMEw9yP0Tk_dRxLfyDtoaXjCp5HJeOQ7Y603CKbtNQQtWqLKjyP5RxL4xj-T_t9Rq_0M"
create_time {
  seconds: 1653951391
  nanos: 773162000
}
update_time {
  seconds: 1653951392
  nanos: 414257000
}
network: "projects/841506577075/global/networks/spotifind-vpc2"

In [6]:
# (IndexService) get ANN index for current region (~us-west1c/us-central1a)
index_service_client = aiplatform_v1.IndexServiceClient(client_options=dict(api_endpoint=aiplatform_endpoint))

list_indexes_request = aiplatform_v1.ListIndexesRequest(parent=location)

In [7]:
indexes_pager_result = index_service_client.list_indexes(request=list_indexes_request)

In [8]:
spotifind_index = indexes_pager_result.indexes[0]
spotifind_index

name: "projects/841506577075/locations/us-west1/indexes/8335635144681455616"
display_name: "spotifind_ann_index"
description: "Spotifind ANN index"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml"
metadata {
  struct_value {
    fields {
      key: "config"
      value {
        struct_value {
          fields {
            key: "algorithmConfig"
            value {
              struct_value {
                fields {
                  key: "treeAhConfig"
                  value {
                    struct_value {
                      fields {
                        key: "leafNodeEmbeddingCount"
                        value {
                          string_value: "500"
                        }
                      }
                      fields {
                        key: "leafNodesToSearchPercent"
                        value {
                          number_value: 7.0
                        }
       

In [9]:
# Get deployed index from index

def deploy_index(index, index_endpoint, client):
    deployed_index = aiplatform_v1.DeployedIndex({
        "id": "{}-deployed".format(index.display_name),
        "index": index.name,
        "display_name": "{}-deployed".format(index.display_name),
        "reserved_ip_ranges": [
            PEERING_RANGE_NAME
        ]
    })
    
    deployed_index_request = aiplatform_v1.DeployIndexRequest(index_endpoint=index_endpoint.name, deployed_index=deployed_index)
    
    return client.deploy_index(request=deployed_index_request)
    

In [10]:
spotifind_index.name

'projects/841506577075/locations/us-west1/indexes/8335635144681455616'

In [11]:
spotifind_index_endpoint.name

'projects/841506577075/locations/us-west1/indexEndpoints/5298379408485122048'

In [12]:
deployed_indexes = spotifind_index.deployed_indexes

deployed_index = None
if not deployed_indexes:
    deployed_index = deploy_index(spotifind_index, spotifind_index_endpoint, index_endpoint_service_client)
    print(deployed_index)
else:
    deployed_index = spotifind_index.deployed_indexes[0]

In [13]:
pager_result = index_endpoint_service_client.list_index_endpoints(request=list_index_endpoints_request)

In [14]:
pager_result.index_endpoints[0].deployed_indexes

[id: "spotifind_ann_index-deployed"
index: "projects/841506577075/locations/us-west1/indexes/8335635144681455616"
display_name: "spotifind_ann_index-deployed"
create_time {
  seconds: 1655663657
  nanos: 887493000
}
private_endpoints {
  match_grpc_address: "172.24.2.5"
}
index_sync_time {
  seconds: 1655683243
  nanos: 368194000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
reserved_ip_ranges: "spotifind-vpc-peering-range2"
deployment_group: "default"
]

In [15]:
# Get gRPC address from private index endpoint for client calls
get_grpc_address = lambda deployed_index: deployed_index.private_endpoints.match_grpc_address
deployed_index = spotifind_index_endpoint.deployed_indexes[0]

In [ ]:
# Match service wrapper init
# 1. (IndexEndpointService) get ANN index endpoint for current region (~us-west1c/us-central1a)
#   https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.index_endpoint_service.IndexEndpointServiceClient#google_cloud_aiplatform_v1_services_index_endpoint_service_IndexEndpointServiceClient_list_index_endpoints
# 2. Get deployed index from index endpoint
#   https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.DeployedIndex
# 3. If deployed index does not exist:
#   a. (IndexService) get ANN index for current region (~us-west1c/us-central1a)
#     https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.index_endpoint_service.IndexEndpointServiceClient#google_cloud_aiplatform_v1_services_index_endpoint_service_IndexEndpointServiceClient_list_index_endpoints
#   b. Deploy index from ANN index endpoint, ANN index, index endpoint service client
#     https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.index_endpoint_service.IndexEndpointServiceClient#google_cloud_aiplatform_v1_services_index_endpoint_service_IndexEndpointServiceClient_deploy_index
# 4. Get gRPC address from private index endpoint for client calls (https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.IndexPrivateEndpoints)
# https://github.com/googleapis/google-cloud-python
# https://github.com/googleapis/python-aiplatform

In [16]:
DEPLOYED_INDEX_SERVER_IP = get_grpc_address(deployed_index)
DEPLOYED_INDEX_ID = deployed_index.id

In [17]:
print(DEPLOYED_INDEX_SERVER_IP)
print(DEPLOYED_INDEX_ID)

172.24.2.5
spotifind_ann_index-deployed


In [18]:
channel = grpc.insecure_channel("{}:10000".format(DEPLOYED_INDEX_SERVER_IP))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

# Test query
query = [
    -0.11333,
    0.48402,
    0.090771,
    -0.22439,
    6.034206,
    -7.55831,
    0.041849,
    -0.53573,
    0.18809,
    -0.58722,
    -1.015313
]

request = match_service_pb2.MatchRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for val in query:
    request.float_val.append(val)

response = stub.Match(request)
response

neighbor {
  id: "2TRu7dMps7cVKOyazkj9Fb"
  distance: -57.44913101196289
}
neighbor {
  id: "0bqrFwY1HixfnusFxhYbDl"
  distance: -60.509368896484375
}
neighbor {
  id: "4BHSjbYylfOH5WAGusDyni"
  distance: -64.6940689086914
}
neighbor {
  id: "3s9f1LQ6607eDj9UYCzmgk"
  distance: -65.47064971923828
}
neighbor {
  id: "2HbKqm4o0w5wEeEFXm2sD4"
  distance: -65.67963409423828
}
neighbor {
  id: "53LkLW4k4w7EOayzvx70O9"
  distance: -65.73475646972656
}
neighbor {
  id: "6vy95NfrMeUfaXpaTm4O53"
  distance: -67.0295181274414
}
neighbor {
  id: "2VQc9orzwE6a5qFfy54P6e"
  distance: -67.75544738769531
}
neighbor {
  id: "6rladQVZPz1K5AgocaUgop"
  distance: -67.8094253540039
}
neighbor {
  id: "4RzpCjByV1NWUGKVQGuej6"
  distance: -67.86714172363281
}

['test']